In [10]:
import pandas as pd
import random
import numpy as np
import pyodbc
from datetime import date, datetime
import openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException

import re 
# тут путь к входному файлу
data = pd.read_excel(r"/Users/allastepannikova/Downloads/ЛВС2.xlsx", dtype="object")
lists = data['ИНН'].to_list()
guids = data['GUID'].to_list()
inn_list = []
guid_list = []
browser = webdriver.Chrome()
browser.get('https://focus.kontur.ru/login')
authorize = browser.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[2]/div/div[2]/div/div/form/div/div[1]/div/div[2]/div/label/span[2]/input')
authorize.send_keys('morozova.anna@ast-ins.ru')
# Vasilyeva.Anna@spb.astcompany.ru
# lozinskay.anna@astcompany.ru morozova.anna@ast-ins.ru
authorize = browser.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[2]/div/div[2]/div/div/form/div/div[2]/div/span/button')
authorize.click()
# тут количество секунд на авторизацию 300 сек=5 минут
time.sleep(7)
for i,el in enumerate(lists):
    if el!=' ':
        inn_list.append(el)
        guid_list.append(guids[i])
url='https://focus.kontur.ru/search?country=RU&query='
soups=[]
inns=[]
guids=[]
# тут ограничения - с какого инн начинаем на каком заканчиваем
for iteration, inn in enumerate(inn_list):
    try:
        current_inn = str(round(inn))
    except:
        current_inn = str(inn)
    try:
        browser.get(url+current_inn)
        time.sleep(5)
        soup = BeautifulSoup(browser.page_source,'html.parser')
        soups.append(soup)
        inns.append(current_inn)
        guids.append(guid_list[iteration])
    except:
        continue
browser.quit() #закрытие окна браузера

In [11]:
names=[]
phones=[]
sites=[]
deyatels=[]
addresses=[]
for inn in range(len(soups)):
    site = ''
    phone=''
    address=''
    deyatel=''
    name=''
    try:
        name = soups[inn].find('h1', class_="_328c-").text
    except:
        name = 'не существует'
    info = soups[inn].find_all('div', class_="_3ZT-C")
    try:
        address = soups[inn].find('div', itemprop="address").text.split('\xa0\u200b\xa0')[0][:6]+ ', '+ soups[inn].find('span', itemprop="addressRegion").text +' '+ soups[inn].find('span', itemprop="streetAddress").text
    except:
        address = ''
    for i,inf in enumerate(info):
        s = inf.text
        if 'постановки на учет' in s:
            try:
                phone =info[i+2].find('span',class_="_2DJgz Ps02p").text
            except:
                phone = ''
            try:
                site =info[i+1].find('span',class_="react-ui-link-text react-ui-xagboe").text
            except:
                site = ''
            if site.startswith('+7'):
                phone=site
                site=''
            if 'Добавить номер' in site:
                site=''
                phone=''
        if 'Уставный капитал' in s:
            try:
                deyatel=info[i-1].find('div',class_="_2NhZ4 _379sV").text.split('\xa0—\xa077')[0].replace('Все виды деятельности','')
            except:
                deyatel=''
            
    names.append(name)
    phones.append(phone)
    sites.append(site)
    deyatels.append(deyatel)
    addresses.append(address)
        
    
# переменная inn_list guid_list - c теми же ограничениями как в перво ячейке. если парсите с 300 по 600 то - inn_list[300:600] guid_list[300:600]
matrix = np.array([names, inns, guids ,phones, sites,deyatels,addresses])
matrix = matrix.transpose()
df = pd.DataFrame(data=(matrix), columns=['Компания','ИНН','GUID','Телефон','Сайт','Сфера деятельности', 'Адрес'])

,Компания,ИНН,GUID,Телефон,Сайт,Сфера деятельности,Адрес
0,"ООО ""А.С. ВОЛГА ЛТД""",6163221783,1497a5cd-d314-47e9-b5de-c695c915c213,,,"Торговля оптовая зерном, необработанным табако...","344022, Ростовская обл ул Суворова, 91"
1,"ООО ""ГРАНД ОПТИКС""",7734510826,0ee1fcdc-38c3-4151-8328-8cfa1a588e94,+7 (495) 737-08-40,grand-optics.ru,"Торговля оптовая изделиями, применяемыми в мед...","127030, г Москва ул Образцова, 3"
2,"ООО ""Судоходная компания ""Донмастер""",6163096638,0a2f2c86-716a-4232-92bc-9c7c7233b635,+7 (863) 261-80-70 5,,Деятельность морского пассажирского транспорта...,"344082, Ростовская обл ул Береговая, 8"
3,"ООО ""ДОНМАСТЕР""",6163057300,c99290da-7e08-49a8-b031-3574b588cf8d,+7 (863) 261-80-70 5,,Деятельность морского грузового транспорта — 5...,"344082, Ростовская обл ул Береговая, 8"
4,"ООО ""ДОНМАСТЕР ПЛЮС""",6164104553,93e4de69-4aa9-42a2-b452-6c7071f78d8c,+7 (863) 261-80-70 5,,Аренда и управление собственным или арендованн...,"344082, Ростовская обл ул Береговая, 8"
...,...,...,...,...,...,...,...
148,"ООО ""ЮНИКОН""",7723506560,f3ce900c-a043-4092-a13f-b0cef5a7025f,+7 (495) 672-18-02,inn-techno.ru,Управление эксплуатацией нежилого фонда за воз...,"111141, г Москва ул Плеханова, 7"
149,"ООО ""УМ и СТ""",7724725614,9f6b9d92-bc90-45ac-82ab-8a23cb77eb33,+7 903 215-26-59 1,,Деятельность автомобильного грузового транспор...,"115477, г Москва ул Кантемировская, 19"
150,"ООО ""Фирма ""Д и С""",5038023782,c765c7a9-a9c5-48e9-9c63-0ad5d99fe714,+7 (495) 741-25-10,firma-dis.ru,Торговля оптовая химическими продуктами — 46.7...,"141282, Московская обл ул Толмачева, 80 / корп 2"
151,"ООО НПО ""ЭКОМАШГРУПП""",6950109310,309d2db1-4e03-44e5-bc53-6fe9ef65759e,+7 (4822) 77-76-04 3,,Производство подъемно-транспортного оборудован...,"170017, Тверская обл ул Коняевская, 23"


In [12]:
# тут путь к выходному файлу - можно менять на любое название (например - первые 300, или с 700 по 1000 и тд)
path=r"/Users/allastepannikova/Downloads/itog_Черваков1.xlsx" 
df.to_excel(path,index=False)